## Übung Machine Learning 1: Voting Classifier

Lernziel: Schreiben eines eigenen, sklearn-kompatiblen Klassifikators

Hinweis des Studenten: Die Tests wurden etwas 'verschönert'. In die Funktionsweise wurde jedoch nicht eingegriffen. Anstatt die doch recht unansehnlichen Fehlermeldungen anzuzeigen, habe ich mit try und catch die Tests um ein Detail verschönert.

### Imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.exceptions import NotFittedError

### Vorbereitung: Laden der Daten

In [2]:
data = pd.read_csv("./DATA/heart.csv")
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


### Vorbereitung: Trainieren unterschiedlicher Klassifikatoren auf den Daten 

In [3]:
# Splitting into test and training data
y = data['target']
X = data.drop('target', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier


# Naive Bayes
clf1 = GaussianNB()

# Decision Tree
clf2 = DecisionTreeClassifier()

# KNN Classifier
clf3 = KNeighborsClassifier()

# Gradient Boosting Classifier
clf4 = GradientBoostingClassifier()

# Random Forest Classifier
clf5 = RandomForestClassifier()

# Because scaling is important for some of the classifiers, we have to do it
# (It doesn't hurt if we do it for all of them)
std_scaler = StandardScaler()


In [5]:
def cross_validate(clf, label, X_train, y_train): 
    print(label)
    print("\nCROSS VALIDATION RESULTS")
    scores = cross_val_score(clf, X_train, y_train, scoring = "accuracy", cv = 10)
    print("Accuracy:", np.round(scores,2))
    print("Mean:", np.round(scores.mean(),2))
    print("Standard deviation:", np.round(scores.std(),2))

def evaluation(clf, X_train, y_train, X_test, y_test):     
    print("\nACCURACY ON TEST DATA")
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(np.round(metrics.accuracy_score(y_test, pred),2))
    print("\n-------------------\n")


In [6]:
classifiers = [('Naive Bayes', clf1), ('Decision Tree', clf2), ('kNN', clf3), ('GradientBoosting', clf4), ('RandomForest', clf5)]

for label, clf in classifiers: 
    pipe = Pipeline([
        ('scaler', std_scaler), 
        ('classifier', clf)
    ])
    cross_validate(pipe, label, X_train, y_train)
    evaluation(pipe, X_train, y_train, X_test, y_test)

Naive Bayes

CROSS VALIDATION RESULTS
Accuracy: [0.88 0.72 0.75 0.67 0.96 0.88 0.83 0.79 0.79 0.83]
Mean: 0.81
Standard deviation: 0.08

ACCURACY ON TEST DATA
0.87

-------------------

Decision Tree

CROSS VALIDATION RESULTS
Accuracy: [0.8  0.56 0.79 0.71 0.79 0.83 0.88 0.83 0.54 0.83]
Mean: 0.76
Standard deviation: 0.11

ACCURACY ON TEST DATA
0.75

-------------------

kNN

CROSS VALIDATION RESULTS
Accuracy: [0.8  0.76 0.88 0.58 0.92 0.88 0.83 0.88 0.71 0.88]
Mean: 0.81
Standard deviation: 0.1

ACCURACY ON TEST DATA
0.87

-------------------

GradientBoosting

CROSS VALIDATION RESULTS
Accuracy: [0.88 0.64 0.71 0.75 0.88 0.79 0.83 0.88 0.71 0.92]
Mean: 0.8
Standard deviation: 0.09

ACCURACY ON TEST DATA
0.77

-------------------

RandomForest

CROSS VALIDATION RESULTS
Accuracy: [0.96 0.64 0.83 0.75 0.92 0.83 0.83 0.88 0.75 0.88]
Mean: 0.83
Standard deviation: 0.09

ACCURACY ON TEST DATA
0.84

-------------------



---------------

## Aufgabe 1: Implementierung eines Maximum Voting Classifiers (17 Punkte)

Implementieren Sie einen Voting Classifier. Bitte beachten Sie dabei folgendes: 
* Verwenden Sie für Ihre Implementierung das untenstehende Template. 
* Ihr Klassifikator soll scikit-learn kompatibel sein. Beachten Sie daher die Hinweise im beigefügten PDF. (Detailliertere Informationen finden Sie hier: https://scikit-learn.org/stable/developers/develop.html, wobei diese Dokumentation viel weiter in die Tiefe geht, als wir es für die Übung brauchen.) Sie finden außerdem ganz unten in diesem Notebook ein Beispiel für eine Implementierung eines Classifiers in diesem Template.  
* Eine Liste mit den zu verwendeten Klassifikatoren soll dem VotingClassifier bei der Initialisierung übergeben werden.
* Es soll möglich sein beim Aufruf zwischen einem Soft Voting und Hard Voting zu wählen (auch dies wird bei der Initialisierung festgelegt, wobei der entsprechende Parameter die beiden Ausprägungen "hard" oder "soft" annehmen kann.)


Der Klassifikator soll robust sein, d.h. mit einigen möglichen Fehlerfällen umgehen können. Dazu gehört unter anderem: 
* Wird ein anderer Wert als "soft" oder "hard" übergeben, soll ein ``ValueError`` geworfen werden mit dem Text: "voting must be 'hard' or 'soft'; got X" (wobei X der übergebene Wert ist).
* Wenn die Liste der Klassifikatoren = None ist oder sie leer ist, soll ebenfalls ein ``ValueError`` geworfen werden. Dieses Mal mit dem Text "At least 1 classifier must be submitted".
* Die Methode predict() darf nicht aufgerufen werden, wenn nicht zuvor fit() aufgerufen worden ist. Dies muss in der predict()-Methode mit einer für Ihren Classifier geeignete Methode überprüft werden und ggf. ein ``NotFittedError`` geworfen werden.
* Es muss sichergestellt werden, dass X und y in fit() die gleiche Anzahl Instanzen haben. Ist dies nicht der Fall, so soll ein ``ValueError`` geworfen werden.


Tipp: Gehen Sie bei der Bearbeitung der Aufgabe folgendermaßen vor: 
* Stellen Sie zunächst sicher, dass Sie wissen, wie ein VotingClassifier funktioniert (siehe Vorlesungsfolien)
* Überlegen Sie anschließend, welche Schritte in welchem Teil des Templates durchgeführt werden müssen (notieren Sie sich diese ggf. kurz in Kommentaren an der entsprechenden Stelle im Code-Template)
* Die für die Robustheit erforderlichen Punkte können in einem 2. Schritt bearbeitet werden. D.h. es ist möglich, dass Sie zunächst die Grundfunktionalität des Codes implementieren und testen. Test 1 und 2 unten sollten anschließend laufen. 
* Führen Sie dann die für die Robustheit erforderlichen Erweiterungen durch und testen Sie diese mit Test 3-7 unten. Tipp: Hier können Sie zum Teil von scikit-learn bereitgestellte Testmethoden verwenden. Beachten Sie hierzu die Hinweise im beigefügten PDF und informieren Sie sich über die entsprechenden Methoden. Überlegen Sie sich jeweils, wo (in welcher Methode) sie die Validierungen jeweils vornehmen müssen. Ggf. müssen die Methoden auch anders parametrisiert werden, als in den Beispielen zu sehen. 
* Machen Sie anschließend weiter mit Aufgabe 2

In [7]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels

class MajorityVoteClassifier(BaseEstimator, ClassifierMixin): 
    
    
    def __init__(self, classifiers = [DecisionTreeClassifier()], voting='hard'):
        """ Initialization
        """        
        self.classifiers = classifiers
        self.voting = voting  
        
        
    def fit(self, X, y):
        """ Fit classifiers.  
        
        Parameters
        ----------
        X : {array-like, sparse matrix},
            shape = [n_examples, n_features]
            Matrix of training examples.
        
        y : array-like, shape = [n_examples]
            Vector of target class labels.
        
        Returns
        -------
        self : object
        
        """
        # -> classifier und voting prüfen
        X, y = check_X_y(X, y)
        # -> x,y gleiche anzahl
        if self.voting not in ['hard', 'soft']:
            raise ValueError("voting must be 'hard' or 'soft'; got "+str(self.voting))
            
        if self.classifiers == None or len(self.classifiers) < 1:
            raise ValueError("At least 1 classifier must be submitted")
            
        for model in self.classifiers:
            try:
                model.fit(X, y)
            except:
                raise ValueError("There is something wrong with the classifiers!")
            
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        return self
    
    
    # nicht nur für 2 Klassen programmiert worden :)
    def predict(self, X):
        """ Predict class labels for X.
        
        Parameters
        ----------
        X : {array-like, sparse matrix},
            Shape = [n_examples, n_features]
            Matrix of training examples.
        
        Returns
        ----------
        maj_vote : array-like, shape = [n_examples]
            Predicted class labels.
        
        """
        check_is_fitted(self, ['X_', 'y_'])
        result = []
        predictions = []
        if self.voting == 'hard':
            # Funktionsweise:
            # 1. Lass die Classifier ihre Vorhersage treffen
            # 2. Summiere die einzelnen Vorhersagen von jedem Classifier(für jeden Datenpunkt)
            # 3. Finde die Klasse, die insgesamt am öftesten Vorhergesagt wurde (wieder für jeden Datenpunkt)
            
            # Hinweis: der Code wird durch 2 Faktoren verkompliziert:
            #    - Zum Einen gibt es mehrere Datenpunkte. Man muss also insgesamt mehrere Objekte klassifizieren.
            #      Im Code muss man deswegen alle Spalten durchgehen. Jede Spalte stellt in diesem Fall einen 
            #      Datenpunkt da. Außerdem muss man die Ergebnisse für jeden Datenpunkt getrennt abspeichern. 
            #      Warum? Naja die Vorhersagen gelten natürlich immer nur für einen Datenpunkt.
            #      Die Resultate einer Vorhersage muss man deswegen für jeden Datenpunkt einzeln abspeichern.
            #      Und das kann man in einer Liste am besten realisieren.
            #    - Zum Anderen habe ich den Code für eine beliebige Anzahl von Klassen geschrieben.
            #      Mein Code ist deshalb etwas abstrakter gehalten, aber eigentlich auch nicht viel komplizierter.
            
            # 1. Lass die Classifier ihre Vorhersage treffen
            for model in self.classifiers:
                result += [model.predict(X).tolist()]
                
            # 2. Summiere die einzelnen Vorhersagen von jedem Classifier(für jeden Datenpunkt)
            for column in range(len(result[0])):
                for row in range(len(result)):
                    # sort result
                    voting = dict()
                    vote = result[row][column]
                    if voting.get(vote) == None:
                        voting[vote] = 1
                    else:
                        voting[vote] += 1
                        
                # 3. Finde die Klasse, die insgesamt am öftesten Vorhergesagt wurde (wieder für jeden Datenpunkt)
                pred = None
                for v in voting.items():
                    if pred == None:
                        pred = v[0]
                    elif v[1] > voting[pred]:
                        pred = v[0]
                predictions += [pred]
            return predictions
        
        elif self.voting == 'soft':
            # Funktionsweise:
            # 1. Lass die Classifier ihre vorhersage treffen (als Wahrscheinlichkeit)
            # 2. Summiere die einzelnen Wahrscheinlichkeiten (für jeden Datenpunkt)
            # 3. Finde die Klasse, die insgesamt die höchste Wahrscheinlichkeit hat (wieder für jeden Datenpunkt)
            
            # (wie auch oben)
            # Hinweis: der Code wird durch 2 Faktoren verkompliziert:
            #    - Zum Einen gibt es mehrere Datenpunkte. Man muss also insgesamt mehrere Objekte klassifizieren.
            #      Im Code muss man deswegen alle Spalten durchgehen. Jede Spalte stellt in diesem Fall einen 
            #      Datenpunkt da. Außerdem muss man die Ergebnisse für jeden Datenpunkt getrennt abspeichern. 
            #      Warum? Naja die Vorhersagen gelten natürlich immer nur für einen Datenpunkt.
            #      Die Resultate einer Vorhersage muss man deswegen für jeden Datenpunkt einzeln abspeichern.
            #      Und das kann man in einer Liste am besten realisieren.
            #    - Zum Anderen habe ich den Code für eine beliebige Anzahl von Klassen geschrieben.
            #      Mein Code ist deshalb etwas abstrakter gehalten, aber eigentlich auch nicht viel komplizierter.
            
            # 1. Lass die Classifier ihre vorhersage treffen (als Wahrscheinlichkeit)
            for model in self.classifiers:
                result += [model.predict_proba(X).tolist()]
                
            # 2. Summiere die einzelnen Wahrscheinlichkeiten (für jeden Datenpunkt)
            pred_sum_collection = []
            for column in range(len(result[0])):
                pred_sum = dict()
                for row in range(len(result)):
                    for class_pos in range(len(result[row][column])):
                        if pred_sum.get(class_pos) == None:
                            pred_sum[class_pos] = result[row][column][class_pos]
                        else:
                            pred_sum[class_pos] += result[row][column][class_pos]
                pred_sum_collection += [pred_sum]
                
            # 3. Finde die Klasse, die insgesamt die höchste Wahrscheinlichkeit hat (wieder für jeden Datenpunkt)
            pred_collection = []
            # Gehe die summierten Wahrscheinlichkeiten der einzelnen Datenpunkte durch
            for column in pred_sum_collection:
                # Berechne die zu vorhersagende Klasse für diesen Datenpunkt
                pred = None
                for p in column.items():
                    if pred == None:
                        pred = p[0]
                    elif p[1] > column[pred]:
                        pred = p[0]
                # Füge die nun gefundene wahrscheinlichste Klasse zu den Ergebnissen
                pred_collection += [pred]
            return pred_collection
    


### Test 1
Wenden Sie Ihren MajorityVoteClassifier mit voting = "hard" an. Führen Sie eine Cross-Validation durch und bestimmen Sie außerdem die Accuracy auf den Testdaten.  

In [8]:
ensemble = [clf1, clf2, clf3, clf4, clf5]
my_clf = MajorityVoteClassifier(ensemble, voting = "hard")
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf)
])
cross_validate(pipe, "voting_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)

voting_clf

CROSS VALIDATION RESULTS
Accuracy: [0.96 0.72 0.88 0.75 0.96 0.79 0.79 0.88 0.79 0.88]
Mean: 0.84
Standard deviation: 0.08

ACCURACY ON TEST DATA
0.84

-------------------



### Test 2
Ändern Sie den Parameter Voting auf "soft". Führen Sie erneut eine Cross-Validation durch und bestimmen Sie außerdem die Accuracy auf den Testdaten.  

In [9]:
my_clf_soft = MajorityVoteClassifier(ensemble, voting = "soft")
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf_soft)    
])

cross_validate(pipe, "voting_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)

voting_clf

CROSS VALIDATION RESULTS
Accuracy: [0.88 0.68 0.83 0.75 0.92 0.79 0.88 0.83 0.71 0.88]
Mean: 0.81
Standard deviation: 0.08

ACCURACY ON TEST DATA
0.82

-------------------



### Test 3
Ihr MajorityVotingClassifier sollte auch mit der Situation umgehen können, dass nur 1 Klassifikator übergeben wird. Stellen Sie sicher, dass Ihr Voting Classifier auch dann funktionsfähig ist und demonstrieren Sie dies hier durch den untenstehenden Aufruf.

In [10]:
ensemble_1 = [clf1]
my_clf = MajorityVoteClassifier(ensemble_1, voting = "hard")
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf)    
])
cross_validate(pipe, "voting_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)

voting_clf

CROSS VALIDATION RESULTS
Accuracy: [0.88 0.72 0.75 0.67 0.96 0.88 0.83 0.79 0.79 0.83]
Mean: 0.81
Standard deviation: 0.08

ACCURACY ON TEST DATA
0.87

-------------------



### Test 4
Ihr MajorityVotingClassifier sollte auch mit der Situation umgehen können, dass eine leere Liste von Klassifikatoren übergeben wird und in diesem Fall die oben definierte Fehlermeldung werfen. Stellen Sie sicher, dass dies der Fall ist und demonstrieren Sie hier die Funktionsweise durch einen Aufruf mit einer leeren Liste.

In [11]:
try:
    ensemble_0 = []
    my_clf = MajorityVoteClassifier(ensemble_0, voting = "hard")
    pipe = Pipeline([
        ('scaler', std_scaler), 
        ('classifier', my_clf)    
    ])
    pipe.fit(X_train, y_train)
    pipe.predict(X_test)
    print("Es wurde leider keine Exception geworfen. Test nicht bestanden!")
except ValueError:
    print("Sehr gut, Test bestanden! :D")

Sehr gut, Test bestanden! :D


### Test 5
Wird ``predict()`` vor ``fit()`` aufgerufen, so soll ein ``NotFittedError`` geworfen werden.  
Stellen Sie sicher, dass dies der Fall ist und demonstrieren Sie hier die Funktionsweise durch einen Aufruf von ``predict()`` ohne vorherigen Aufruf von ``fit()``. 

In [12]:
try:
    my_clf2 = MajorityVoteClassifier(ensemble, voting = "hard")
    pipe = Pipeline([
        ('scaler', std_scaler), 
        ('classifier', my_clf2)
    ])

    pipe.predict(X_test)
    print("Es wurde leider keine Exception geworfen. Test nicht bestanden!")
except NotFittedError:
    print("Sehr gut, Test bestanden! :D")

Sehr gut, Test bestanden! :D


### Test 6
Wenn die Anzahl Zeilen in X nicht mit der Anzahl der Zeilen in y übereinstimmt, soll ein ``ValueError`` geworfen werden. Stellen Sie sicher, dass dies der Fall ist und demonstrieren Sie hier die Funktionsweise durch den unten stehenden Aufruf

In [13]:
try:
    my_clf = MajorityVoteClassifier(ensemble, voting = "hard")
    pipe = Pipeline([
        ('scaler', std_scaler), 
        ('classifier', my_clf)
    ])
    X_train2 = X_train.iloc[3:,:]

    pipe.fit(X_train2, y_train)
    pipe.predict(X_test)
    print("Es wurde leider keine Exception geworfen. Test nicht bestanden!")
except ValueError:
    print("Sehr gut, Test bestanden! :D")

Sehr gut, Test bestanden! :D


### Test 7
Wenn ein anderer String als "hard" oder "soft" übergeben wird, soll eine Fehlermeldung geworfen werden. Demonstrieren Sie durch Aufruf des unten stehenden Codes, dass das bei Ihnen der Fall ist.

In [14]:
try:
    my_clf = MajorityVoteClassifier(ensemble, voting = "hard2")
    pipe = Pipeline([
        ('scaler', std_scaler), 
        ('classifier', my_clf)
    ])
    pipe.fit(X_train, y_train)
    pipe.predict(X_test)
    print("Es wurde leider keine Exception geworfen. Test nicht bestanden!")
except ValueError:
    print("Sehr gut, Test bestanden! :D")

Sehr gut, Test bestanden! :D


## Aufgabe 2 (3 Punkte)
Entfernen Sie den in der Einzelwertung schwächsten Klassifikator (bzw. ggf. auch 2 Klassifikatoren, falls es mehrere gibt, die deutlich schlechter als alle anderen sind) und testen Sie den Voting Classifier erneut. Führen Sie den Test sowohl mit voting = "soft" als auch voting = "hard" durch.

In [15]:
# Neurung kommt weiter unten
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics

class MajorityVoteClassifier(BaseEstimator, ClassifierMixin): 
    
    
    def __init__(self, classifiers = [DecisionTreeClassifier()], voting='hard'):
        """ Initialization
        """        
        self.classifiers = classifiers
        self.voting = voting  
        
        
    def fit(self, X, y):
        """ Fit classifiers.  
        
        Parameters
        ----------
        X : {array-like, sparse matrix},
            shape = [n_examples, n_features]
            Matrix of training examples.
        
        y : array-like, shape = [n_examples]
            Vector of target class labels.
        
        Returns
        -------
        self : object
        
        """
        # -> classifier und voting prüfen
        X, y = check_X_y(X, y)
        # -> x,y gleiche anzahl
        if self.voting not in ['hard', 'soft']:
            raise ValueError("voting must be 'hard' or 'soft'; got "+str(self.voting))
            
        if self.classifiers == None or len(self.classifiers) < 1:
            raise ValueError("The model needs classifiers to work!")
            
        for model in self.classifiers:
            try:
                model.fit(X, y)
            except:
                raise ValueError("There is something wrong with the classifiers!")
            
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        return self
    
    
    # nicht nur für 2 Klassen programmiert worden :)
    def predict(self, X):
        """ Predict class labels for X.
        
        Parameters
        ----------
        X : {array-like, sparse matrix},
            Shape = [n_examples, n_features]
            Matrix of training examples.
        
        Returns
        ----------
        maj_vote : array-like, shape = [n_examples]
            Predicted class labels.
        
        """
        # Entferne schwache Classifier:
        check_is_fitted(self, ['X_', 'y_'])
        result = []
        predictions = []
        if self.voting == 'hard':
            # Funktionsweise:
            # 1. Lass die Classifier ihre Vorhersage treffen
            # 2. Summiere die einzelnen Vorhersagen von jedem Classifier(für jeden Datenpunkt)
            # 3. Finde die Klasse, die insgesamt am öftesten Vorhergesagt wurde (wieder für jeden Datenpunkt)
            
            # Hinweis: der Code wird durch 2 Faktoren verkompliziert:
            #    - Zum Einen gibt es mehrere Datenpunkte. Man muss also insgesamt mehrere Objekte klassifizieren.
            #      Im Code muss man deswegen alle Spalten durchgehen. Jede Spalte stellt in diesem Fall einen 
            #      Datenpunkt da. Außerdem muss man die Ergebnisse für jeden Datenpunkt getrennt abspeichern. 
            #      Warum? Naja die Vorhersagen gelten natürlich immer nur für einen Datenpunkt.
            #      Die Resultate einer Vorhersage muss man deswegen für jeden Datenpunkt einzeln abspeichern.
            #      Und das kann man in einer Liste am besten realisieren.
            #    - Zum Anderen habe ich den Code für eine beliebige Anzahl von Klassen geschrieben.
            #      Mein Code ist deshalb etwas abstrakter gehalten, aber eigentlich auch nicht viel komplizierter.
            
            # 1. Lass die Classifier ihre Vorhersage treffen
            for model in self.classifiers:
                result += [model.predict(X).tolist()]
                
            # 2. Summiere die einzelnen Vorhersagen von jedem Classifier(für jeden Datenpunkt)
            for column in range(len(result[0])):
                for row in range(len(result)):
                    # sort result
                    voting = dict()
                    vote = result[row][column]
                    if voting.get(vote) == None:
                        voting[vote] = 1
                    else:
                        voting[vote] += 1
                        
                # 3. Finde die Klasse, die insgesamt am öftesten Vorhergesagt wurde (wieder für jeden Datenpunkt)
                pred = None
                for v in voting.items():
                    if pred == None:
                        pred = v[0]
                    elif v[1] > voting[pred]:
                        pred = v[0]
                predictions += [pred]
            return predictions
        
        elif self.voting == 'soft':
            # Funktionsweise:
            # 1. Lass die Classifier ihre vorhersage treffen (als Wahrscheinlichkeit)
            # 2. Summiere die einzelnen Wahrscheinlichkeiten (für jeden Datenpunkt)
            # 3. Finde die Klasse, die insgesamt die höchste Wahrscheinlichkeit hat (wieder für jeden Datenpunkt)
            
            # (wie auch oben)
            # Hinweis: der Code wird durch 2 Faktoren verkompliziert:
            #    - Zum Einen gibt es mehrere Datenpunkte. Man muss also insgesamt mehrere Objekte klassifizieren.
            #      Im Code muss man deswegen alle Spalten durchgehen. Jede Spalte stellt in diesem Fall einen 
            #      Datenpunkt da. Außerdem muss man die Ergebnisse für jeden Datenpunkt getrennt abspeichern. 
            #      Warum? Naja die Vorhersagen gelten natürlich immer nur für einen Datenpunkt.
            #      Die Resultate einer Vorhersage muss man deswegen für jeden Datenpunkt einzeln abspeichern.
            #      Und das kann man in einer Liste am besten realisieren.
            #    - Zum Anderen habe ich den Code für eine beliebige Anzahl von Klassen geschrieben.
            #      Mein Code ist deshalb etwas abstrakter gehalten, aber eigentlich auch nicht viel komplizierter.
            
            # 1. Lass die Classifier ihre vorhersage treffen (als Wahrscheinlichkeit)
            for model in self.classifiers:
                result += [model.predict_proba(X).tolist()]
                
            # 2. Summiere die einzelnen Wahrscheinlichkeiten (für jeden Datenpunkt)
            pred_sum_collection = []
            for column in range(len(result[0])):
                pred_sum = dict()
                for row in range(len(result)):
                    for class_pos in range(len(result[row][column])):
                        if pred_sum.get(class_pos) == None:
                            pred_sum[class_pos] = result[row][column][class_pos]
                        else:
                            pred_sum[class_pos] += result[row][column][class_pos]
                pred_sum_collection += [pred_sum]
                
            # Entferne 
                
            # 3. Finde die Klasse, die insgesamt die höchste Wahrscheinlichkeit hat (wieder für jeden Datenpunkt)
            pred_collection = []
            # Gehe die summierten Wahrscheinlichkeiten der einzelnen Datenpunkte durch
            for column in pred_sum_collection:
                # Berechne die zu vorhersagende Klasse für diesen Datenpunkt
                pred = None
                for p in column.items():
                    if pred == None:
                        pred = p[0]
                    elif p[1] > column[pred]:
                        pred = p[0]
                # Füge die nun gefundene wahrscheinlichste Klasse zu den Ergebnissen
                pred_collection += [pred]
            return pred_collection
        
        
    # Neu hinzugefügt!!! (Aufgabe 2)
    
    # Helper:
    # berechne die Accuracy für einen Classifier -> wird in remove_weak_classifier verwendet
    def calc_accuracy(self, clf, X_train, y_train, X_test, y_test):  
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        return np.round(metrics.accuracy_score(y_test, pred),2)

    # Helper:
    # berechnet den Median von einer Liste -> ist speziell für die Accuracy angepasst, welche ich so gespeichert habe: (index, accuracy)
    #                                                                                      -> siehe bei remove_weak_classifier
    def calc_median(self, x:list):
        if len(x) > 1:
            if len(x)%2 == 0:
                half1 = len(x)//2
                half2= len(x)//2 -1
                return (x[half1][1] + x[half2][1])/2
            else:
                half = len(x)//2 + 1
                return x[half][1]

    # Entfernt schwache Classifier
    # wird vor predict und vor fit aufgerufen -> siehe die Tests weiter unten
    def remove_weak_classifier(self, X_train, y_train, X_test, y_test):
        # Berechne die Accuracy von jedem classifier
        accuracy = []
        counter = 0
        for classifier in self.classifiers:
            accuracy += [(counter, self.calc_accuracy(classifier, X_train, y_train, X_test, y_test))]
            #                       -> Ich speicher mir noch den Index von dem Classifier ab, damit ich noch weis, zu wem die Accuracy gehört
            #                          Ich sortiere nämlich gleich die Liste
            counter += 1

        # Brechne den Median -> damit man Abweichungen feststellen kann
        # muss man nicht machen, bzw. man könnte es auch anders lösen
        median = self.calc_median(accuracy)
        
        # Schwächste Accuracies ausfindig machen -> vorne die schwächsten und nach hinten immer stärker
        # lambda x:x[1], da accuracy eine Liste von Tupel ist [(index, accuracy), ()...]
        # und wir wollen hier nach der accuracy sortieren: die liegt auf dem index 1, wenn man ein Element anschaut
        accuracy.sort(key=lambda x:x[1], reverse=False)
        
        # Je nach Länge: prüfe ob kleiner als Median -> man hätte sie auch einfach direkt ohne überprüfung entfernen können
        to_remove = []
        if len(accuracy) > 1 and len(accuracy) <= 4:
            if accuracy[0][1] < median:
                    to_remove += [accuracy[0][0]]
        elif len(accuracy) >= 5:
            for acc in range(2):
                # Man hätte auch direkt die letzten beiden entfernen können
                # oder eine anderes Ausschusskriterium wählen können (nicht unbedingt median)
                if accuracy[acc][1] < median:
                    to_remove += [accuracy[acc][0]]
                    
        # Entferne nun die gerade gefundenen schwachen Classifier             
        new_classifiers = []
        for i, x in enumerate(self.classifiers):
            if i in to_remove:
                # ne, den wollen wir nicht in unsere Liste aufnehmen
                continue
            new_classifiers += [x]
        print("\n-----\nOld_Classifiers", self.classifiers)
        print("New_Classifiers", new_classifiers)
        print("accuracies:", accuracy)
        print("median:", median,"\n-----\n")
        self.classifiers = new_classifiers
                            

In [16]:

my_clf = MajorityVoteClassifier(ensemble, voting = "hard")
my_clf.remove_weak_classifier(X_train, y_train, X_test, y_test)
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf)
])
cross_validate(pipe, "voting_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)


-----
Old_Classifiers [GaussianNB(), DecisionTreeClassifier(), KNeighborsClassifier(), GradientBoostingClassifier(), RandomForestClassifier()]
New_Classifiers [GaussianNB(), GradientBoostingClassifier(), RandomForestClassifier()]
accuracies: [(2, 0.7), (1, 0.75), (3, 0.79), (4, 0.84), (0, 0.87)]
median: 0.79 
-----

voting_clf

CROSS VALIDATION RESULTS
Accuracy: [0.96 0.64 0.88 0.75 0.96 0.83 0.79 0.88 0.79 0.88]
Mean: 0.84
Standard deviation: 0.09

ACCURACY ON TEST DATA
0.82

-------------------



In [17]:
my_clf = MajorityVoteClassifier(ensemble, voting = "soft")
my_clf.remove_weak_classifier(X_train, y_train, X_test, y_test)
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf)
])
cross_validate(pipe, "voting_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)


-----
Old_Classifiers [GaussianNB(), DecisionTreeClassifier(), KNeighborsClassifier(), GradientBoostingClassifier(), RandomForestClassifier()]
New_Classifiers [GaussianNB(), GradientBoostingClassifier(), RandomForestClassifier()]
accuracies: [(2, 0.7), (1, 0.72), (3, 0.79), (4, 0.82), (0, 0.87)]
median: 0.79 
-----

voting_clf

CROSS VALIDATION RESULTS
Accuracy: [0.92 0.76 0.83 0.79 0.88 0.83 0.79 0.83 0.75 0.88]
Mean: 0.83
Standard deviation: 0.05

ACCURACY ON TEST DATA
0.87

-------------------



In [18]:
ensemble_1 = [clf1]
my_clf = MajorityVoteClassifier(ensemble_1, voting = "hard")
my_clf.remove_weak_classifier(X_train, y_train, X_test, y_test)
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf)    
])
cross_validate(pipe, "voting_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)


-----
Old_Classifiers [GaussianNB()]
New_Classifiers [GaussianNB()]
accuracies: [(0, 0.87)]
median: None 
-----

voting_clf

CROSS VALIDATION RESULTS
Accuracy: [0.88 0.72 0.75 0.67 0.96 0.88 0.83 0.79 0.79 0.83]
Mean: 0.81
Standard deviation: 0.08

ACCURACY ON TEST DATA
0.87

-------------------



In [19]:
ensemble = [clf1, clf2, clf4, clf5]
my_clf = MajorityVoteClassifier(ensemble, voting = "hard")
my_clf.remove_weak_classifier(X_train, y_train, X_test, y_test)
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf)
])
cross_validate(pipe, "voting_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)


-----
Old_Classifiers [GaussianNB(), DecisionTreeClassifier(), GradientBoostingClassifier(), RandomForestClassifier()]
New_Classifiers [GaussianNB(), GradientBoostingClassifier(), RandomForestClassifier()]
accuracies: [(1, 0.75), (2, 0.77), (3, 0.8), (0, 0.87)]
median: 0.76 
-----

voting_clf

CROSS VALIDATION RESULTS
Accuracy: [0.92 0.68 0.88 0.75 0.92 0.79 0.79 0.79 0.79 0.88]
Mean: 0.82
Standard deviation: 0.07

ACCURACY ON TEST DATA
0.82

-------------------



## Abschlussaufgabe (unbewertet)
Vergleichen Sie nun die Ergebnisse aller Tests. Womit konnten Sie die besten Ergebnisse erzielen? Wie stabil und somit belastbar sind die Ergebnisse? Was könnten nächste Schritte sein, um eine weitere Verbesserung zu erreichen? 

Die Entfgernung schwacher Classifier zeigt eine leichte Verbesserung. Jedoch schwankt die Accuracy natürlich. <br>
Vor allem die Accuracy beim Soft-Voting verbessert sich. <br>
Wenn man im Vorhinein nur Klassifizierer, die für die Situation geeigent sind, wählt, würde man den Klassifizierer schon in die richtige Richtung schieben.<br>
<br>
Als Letztes habe ich bemerkt, dass der VotingClassifier mit weniger Classifiern nicht so stabil ist, wie der mit mehr Classifiern. Das ergibt eigentlich auch Sinn, da das Ergebnis stärker von den einzelnen Ergebnissen abhängig ist (umso weniger Classifier verwendet werden).


--------------------------------

### Offizielles Beispiel-Template für einen scikit-learn kompatiblen Klassifikator

**----------**<br>**Hinweis um sich nicht von diesem Beispiel verwirren zulassen:**<br><br>
Hier wird ein KNN-Klassifizierer realisiert, wobei nur der erste nächste Nachbar zum klassifizieren verwendet wird (k=1).<br>
Der KNN muss die Trainingsdaten nicht lernen, sondern speichert sie und bei der Vorhersage, schaut er einfach, welcher der nächste Nachbar ist und gibt die Klasse von diesem Punkt zurück.<br>
Man sagt auch lazy-learner zu dem KNN. Und genau deswegen ist in der fit-Funktion nichts wirkliches zu sehen.<br>
Die Klassifizieren findet dann bei predict() statt.<br><br>
-> closest = np.argmin(euclidean_distances(X, self.X_), axis=1)<br>
->        return self.y_[closest]<br>
<br>
Diese beiden Zeilen finden den nächsten Nachbarn und geben die Klasse von diesem Punkt zurück.<br>
**----------**

In [20]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances

class TemplateClassifier(ClassifierMixin, BaseEstimator):
    """ An example classifier which implements a 1-NN algorithm.
    For more information regarding how to build your own classifier, read more
    in the :ref:`User Guide <user_guide>`.
    Parameters
    ----------
    demo_param : str, default='demo'
        A parameter used for demonstation of how to pass and store paramters.
    Attributes
    ----------
    X_ : ndarray, shape (n_samples, n_features)
        The input passed during :meth:`fit`.
    y_ : ndarray, shape (n_samples,)
        The labels passed during :meth:`fit`.
    classes_ : ndarray, shape (n_classes,)
        The classes seen at :meth:`fit`.
    """
    def __init__(self, demo_param='demo'):
        self.demo_param = demo_param

    def fit(self, X, y):
        """A reference implementation of a fitting function for a classifier.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The training input samples.
        y : array-like, shape (n_samples,)
            The target values. An array of int.
        Returns
        -------
        self : object
            Returns self.
        """
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        # Return the classifier
        return self

    def predict(self, X):
        """ A reference implementation of a prediction for a classifier.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            The label for each sample is the label of the closest sample
            seen during fit.
        """
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])

        # Input validation
        X = check_array(X)

        closest = np.argmin(euclidean_distances(X, self.X_), axis=1)
        return self.y_[closest]

In [21]:
my_clf = TemplateClassifier()
pipe = Pipeline([
    ('scaler', std_scaler), 
    ('classifier', my_clf)
])
cross_validate(pipe, "knn_clf", X_train, y_train)
evaluation(pipe, X_train, y_train, X_test, y_test)

knn_clf

CROSS VALIDATION RESULTS
Accuracy: [0.76 0.56 0.75 0.62 0.83 0.79 0.79 0.75 0.58 0.83]
Mean: 0.73
Standard deviation: 0.1

ACCURACY ON TEST DATA
0.87

-------------------



--------------